# [How to create a custom Retriever](https://python.langchain.com/docs/how_to/custom_retriever/)

## Interface
To create your own retriever, you need to extend the BaseRetriever class and implement the following methods:

| Method                   | Description                               | Required/Optional |
|--------------------------|-------------------------------------------|--------------------|
| _get_relevant_documents  | Get documents relevant to a query.        | Required          |
| _aget_relevant_documents | Implement to provide async native support.| Optional          |

The logic inside of _get_relevant_documents can involve arbitrary calls to a database or to the web using requests.

#### tip
By inherting from BaseRetriever, your retriever automatically becomes a LangChain Runnable and will gain the standard Runnable functionality out of the box!

In [ ]:
from typing import List

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever


class ToyRetriever(BaseRetriever):
    """A toy retriever that contains the top k documents that contain the user query.

    This retriever only implements the sync method _get_relevant_documents.

    If the retriever were to involve file access or network access, it could benefit
    from a native async implementation of `_aget_relevant_documents`.

    As usual, with Runnables, there's a default async implementation that's provided
    that delegates to the sync implementation running on another thread.
    """

    documents: List[Document]
    """List of documents to retrieve from."""
    k: int
    """Number of top results to return"""

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Sync implementations for retriever."""
        matching_documents = []
        for document in self.documents:
            if len(matching_documents) > self.k:
                return matching_documents

            if query.lower() in document.page_content.lower():
                matching_documents.append(document)
        return matching_documents

    # Optional: Provide a more efficient native implementation by overriding
    # _aget_relevant_documents
    # async def _aget_relevant_documents(
    #     self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    # ) -> List[Document]:
    #     """Asynchronously get documents relevant to a query.

    #     Args:
    #         query: String to find relevant documents for
    #         run_manager: The callbacks handler to use

    #     Returns:
    #         List of relevant documents
    #     """

In [2]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"type": "dog", "trait": "loyalty"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"type": "cat", "trait": "independence"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"type": "fish", "trait": "low maintenance"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"type": "bird", "trait": "intelligence"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"type": "rabbit", "trait": "social"},
    ),
]
retriever = ToyRetriever(documents=documents, k=3)

In [3]:
retriever.invoke("that")

[Document(metadata={'type': 'cat', 'trait': 'independence'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'type': 'rabbit', 'trait': 'social'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [4]:
await retriever.ainvoke("that")

[Document(metadata={'type': 'cat', 'trait': 'independence'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'type': 'rabbit', 'trait': 'social'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [7]:
retriever.invoke("dog")

[Document(metadata={'type': 'dog', 'trait': 'loyalty'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [5]:
retriever.batch(["dog", "cat"])

[[Document(metadata={'type': 'dog', 'trait': 'loyalty'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(metadata={'type': 'cat', 'trait': 'independence'}, page_content='Cats are independent pets that often enjoy their own space.')]]

In [10]:
async for event in retriever.astream_events("bar", version="v1"):
    print(event)

{'event': 'on_retriever_start', 'run_id': '53b9e75f-7048-4158-82c2-f4d66405c44e', 'name': 'ToyRetriever', 'tags': [], 'metadata': {}, 'data': {'input': 'bar'}, 'parent_ids': []}
{'event': 'on_retriever_stream', 'run_id': '53b9e75f-7048-4158-82c2-f4d66405c44e', 'tags': [], 'metadata': {}, 'name': 'ToyRetriever', 'data': {'chunk': []}, 'parent_ids': []}
{'event': 'on_retriever_end', 'name': 'ToyRetriever', 'run_id': '53b9e75f-7048-4158-82c2-f4d66405c44e', 'tags': [], 'metadata': {}, 'data': {'output': []}, 'parent_ids': []}


In [11]:
async for event in retriever.astream_events("dog", version="v1"):
    print(event)

{'event': 'on_retriever_start', 'run_id': '0c59846b-56ce-4c1a-aade-1a7819d62b90', 'name': 'ToyRetriever', 'tags': [], 'metadata': {}, 'data': {'input': 'dog'}, 'parent_ids': []}
{'event': 'on_retriever_stream', 'run_id': '0c59846b-56ce-4c1a-aade-1a7819d62b90', 'tags': [], 'metadata': {}, 'name': 'ToyRetriever', 'data': {'chunk': [Document(metadata={'type': 'dog', 'trait': 'loyalty'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]}, 'parent_ids': []}
{'event': 'on_retriever_end', 'name': 'ToyRetriever', 'run_id': '0c59846b-56ce-4c1a-aade-1a7819d62b90', 'tags': [], 'metadata': {}, 'data': {'output': [Document(metadata={'type': 'dog', 'trait': 'loyalty'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]}, 'parent_ids': []}
